# Pseudorandom generators:

L'idea è quella di ottenere uno schema di encryption <b>EAV-sicuro</b>, basato su OTP, ma con chiavi più corte del plaintext.

Per avere una chiave corta, è necessario espanderla:\
$k \leftarrow \{0,1\}^m \quad \ m<<n \quad \ G(k) = r \in \{0, 1\}^n$

$E_k(x) = r ⊕ k = G(K) ⊕ x$

$G$ è una funzione <b>deterministica</b> che espande $k$: $\quad k$ è il <b>seed</b> di un generatore pseudorandom

Vorrei che $r$ fosse indistinguibile da una sequenza di $n$ bit random (per un attaccante polinomiale)

### Complessità di Kolmogorov:

Una sequenza di bit è <b>random</b> se non esiste un programma che la genera che sia sostanzialmente più corto della sequenza stessa:\
in altre parole, se ho una sequenza e un programma che al genera, se il programam è più corto della sequenza, la sequenza non è random.

Kolmagorov considera anche i programmi <b>inefficienti</b>, noi restringiamo il campo soltanto a quelli <b>efficienti</b>.

### Approccio crittografico:

Non consideriamo sequenza di bit random, ma una <b>distribuzione</b> random di sequenze.\
Non consideriamo, inoltre, i programmi che generano le sequenze: definiamo come random una distribuzione quando supera tutti i possibili test statistici.

## Distinguishers:

Un <b>distinguisher</b> è un <b>algoritmo $PPTIME$</b> che simula i test statistici: prende in input una sequenza di bit e restituisce in output 0 se la sequenza <b>non</b> sembra random oppure 1 se la sequenza in input sembra random.

Un generatore pseudorandom $G$ è buono se il miglior distinguisher può soltanto <b>tirare a indovinare</b>.

## Generatore pseudorandom:

$G$ algoritmo <b>deterministico</b> $PTIME$ da $\{0,1\}^n$ a $\{0,1\}^{l(n)}$

$G$ è un <b> generatore pseudorandom</b> (PRG) se: 

1. $\forall n \ \ l(n) > n \quad$ <i>espansività</i>
2. $Pr(PRG_{M,G}(n) = 1) \leq 1/2 + negl(n) \quad \forall M \in PPTIME$

## Esperimento $PRG_{M,G}(n)$

$1. \quad A: \ \ b \leftarrow \{0,1\}$
- $if \ \ b==0: \quad r \leftarrow \{0,1\}^{l(n)} \quad$ <i>completamente random</i> 
- $if \ \ b==1: \quad s \leftarrow \{0,1\}^{n} \quad r:=G(s) \quad$ <i>soltanto il <b>seed</b> è random, $r$ è <b>deterministico</b></i>

$2. \quad A \rightarrow M: \ \ r$\
$3. \quad M: \ \ b_m \quad$ <i>$M$ deve indovinare $b_m$

$PRG_{M,G}(n) = 1 \ \ se \ \ b_m=b$ 

$M$ deve distinguere <b>randomness</b> ($b=0$) da <b>pseudorandomness</b> ($b=1$)

<b>Attenzione</b>: bisogna controllare che M (ovvero l'attaccante o, in questo caso il <b>distinguisher</b>, sia <b>polinomiale</b> (efficiente) e anche la condizione di <b>espansività</b>

#### Esercizio #1:

$G(s)=sb' \quad b' = ⊕_{i=1...n}^{n} s[i]$

$M: \quad 0 \ \ if \ ⊕_{i=1...n}^{n} r[i] \neq r[n+1] \ \ else: \ \ 1 $

$G$ <b>non</b> è pseudorandom

Praticamente, $M$ sta facendo gli stessi passi di $G$ visto che può vedere il <b>seed</b> (restituito in chiaro) con lo XOR di tutti i bit del seed messo come ultimo.

#### Esercizio #2:

$G(s)=ss$

$l(n)>n \quad$ <i>soddisfa <b>espansività</b></i> 

strategia del distinguisher: $\quad 1 \ \ if \ w[1..n] == w[n+1..2n] \ \ 0 \ \ otherwise$

$Pr(PRG_{D,G}(n)=1) = \frac{1}{2} (Pr(b_D=0 | b=0)Pr(b_D=1 | b=1))$

$Pr(b_D=0|b=0) = 1 - \frac{1}{2^n-2} \quad$ <i>i casi sfortunati dove $r=0^n$ o $r=1^n$</i> \
$Pr(b_D=1|b=1) = 1$ 

$Pr(PRG_{D,G}(n)=1) = \frac{1}{2}(1 - \frac{1}{2^n-2} + 1) > \frac{1}{2} + negl(n)$

In [3]:
import random

def g(n): 
    b = random.randint(0,1)
    r = ''
    s = ''
    
    if(b==0):
        for i in range(2*n):
            r += str(random.randint(0,1))
    else:
        for i in range(n):
            s += str(random.randint(0,1))
        r = s + s

    print("b is equal to: ", b)
    print("R is equal to: ", r)
    print("s is equal to: ", s)

    return r

def distinguisher(r):
    return r[:len(r)//2] != r[-len(r)//2:] # if the two halves are equal, say 0 else 1: this is why != instead of ==

In [8]:
n=10
distinguisher(g(n))

# note: if b==0 and output is True: M wins (because it means G is true that it is a PRG)
# note: if b==1 and output is False: M wins (because it means G is false that it is a PRG)

b is equal to:  0
R is equal to:  10001110100101100001
s is equal to:  


True

### Esercizio #3:

$G(s) = s || f(s)$

$|f(s)| > 0 \quad f \in PTIME$

$D$ (o $M$) conosce la lunghezza di $s$: $n$ e conosce anche la funzione $f$ (principio di Kerckhoffs): ovviamente non può però conoscere l'esito delle generazioni random private di Alice.

$1. \quad A: \ \ b \leftarrow \{0,1\}$
- $if \ \ b==0: \quad r \leftarrow \{0,1\}^{l(n)} \quad$ 
- $if \ \ b==1: \quad s \leftarrow \{0,1\}^{n} \quad r:=G(s) = s||f(s) \quad$ 

$2. \quad A \rightarrow M: \ \ r=w=w_1..w_nw_{n+1}..w_m \quad dove \ \ |w|=m=l(n)$

$w_1..w_n = s$\
$w_{n+1}..w_m = f(s)$\

$3. \quad M: \ \ b_m = \{(w_{n+1}..w_m \ == \ f(w_1..w_n)) \ \ ? \ \ 1 \ \ : \ \  0 \}\quad$ <i>$M$ calcola $f(s)$ perchè, conoscendo $n=|s|$, è in grado di dividere la stringa come vuole</i>

$Pr(b_m=0 | b=0) = Pr((f(w_1..w_n) \neq w_{n+1}..w_m) | randomness) = 1 - \frac{1}{2^{m-n}} \quad$ <i>indovina quasi sempre, tranne nel caso sfortunato dove, per caso, la seconda parte è il risultato della funzione $f$ applicata alla prima parte</i>\
$Pr(b_m=1 | b=1) = 1 \quad$ <i>indovina sempre</i>

$G$ <b>non</b> è pseudorandom

## Pseudo-OTP ($_pOTP$):

Sia $G \in \{0,1\}^n \rightarrow \{0,1\}^{l(n)}$

$_pOTP = (Gen,E_k,D_k)$

$Gen(1^n) = \{k \leftarrow \{0,1\}^n\}$\
$E_k(x) = G(k) ⊕ x$\
$D_k(y) = G(k) ⊕ y$

$x,y \in \{0,1\}^{l(n)}$

## Teorema:

$G$ è $PRG \iff$ $_pOTP \ $ è $\ EAV$-sicuro

### Dimostrazione:

#### Contropposizione logica: 

$A \implies B$ $\quad \quad$ $\neg B \implies \neg A$ 

Hanno la stessa tavola di verità.

#### $(\Leftarrow)$: $\quad$ $_pOTP \ $ è $\ EAV$-sicuro $\Rightarrow G$ è $PRG$

Usiamo la <b>contropposizione logica</b>: $\quad \quad G$ <b>non</b> è $PRG$ $\implies$: $\ _pOTP$ <b>non</b> è $\ EAV$-sicuro

Cosa vuol dire $G$ <b>non</b> è $PRG$? \
$\exists D \in PPTIME : \ \ Pr(PRG_{D,G}(n)=1) > 1/2 + negl(n) $ 

$PrivK^{eav}_{M,\ _pOTP}(n):$

$1. \ \ M \rightarrow A: \quad x_0 \leftarrow \{0,1\}^{l(n)} \quad x_1 = 0^{l(n)} \quad$ <i>$x_0$ è completamente <b>random</b>, $ \ x_1$ no (stesso pattern di $G$)</i>\
$2. \ \ A \rightarrow M: \quad y = E_k(x_b) = x_b ⊕ G(k) \ \ $ con $ \ \ b \leftarrow \{0,1\}$\
$3. \ \ M: \quad b_m = D(y) \quad$ <i>$M$ usa il distinguisher sul ciphertext $y$</i>

$M$ sta usando il distinguisher $D$ che è in grado di "rompere" $G$, e $D \in PPTIME$

se $b=0 \quad$ $y=r ⊕ G(k) \quad$ <i>$r$ è random (è $x_0$)</i>\
se $b=1 \quad$ $y=0^n ⊕ G(k) = G(k)$

$Pr(PrivK^{eav}_{M,\ _pOPT}(n)=1)= \frac{1}{2}(Pr(b_m=0 | b=0) + Pr(b_m=1 | b=1))$

$Pr(b_m=0 | b=0) \ = \ Pr(D(y)=0) \ = \ Pr(D(r⊕G(k))=0) \ = \ Pr(D(r)=0) \quad$ <i>perchè: (random $⊕$ non random) = random</i>\
$Pr(b_m=1 | b=1) \ = \ Pr(D(y)=1) \ = \ Pr(D(G(k))=1) $

$Pr(PrivK^{eav}_{M,\ _pOPT}(n)=1) \ = \ \frac{1}{2}(Pr(D(r)=0) + Pr(D(G(k))=1)) \ = \ \frac{1}{2}(Pr(b_d=0 | b=0) + Pr(b_d=1 | b=1)) \ =  \ Pr(PRG_{D,g}(n)=1)$

$Pr(PrivK^{eav}_{M,\ _pOPT}(n)=1) \ = \ Pr(PRG_{D,g}(n)=1) \ > \ 1/2 + negl(n) \ \ $ perchè $G$ <b>non</b> è PRG $\rightarrow$ $_pOTP$ <b>non è</b> EAV-sicuro

#### $(\Rightarrow)$: $\quad$ $G \ $ è $PRG$ $\Rightarrow$ $_pOTP$ è $EAV$-sicuro

Usiamo la <b>contropposizione logica</b>: $\quad \quad _pOTP$ <b>non</b> è $EAV$-sicuro $\implies$: $\ G$ <b>non</b> è $\ PRG$

Cosa vuol dire $_pOTP$ <b>non</b> è $EAV$-sicuro? \
$\exists M \in PPTIME : \ \ Pr(PrivK^{eav}_{M,\ _pOTP}(n)=1) > 1/2 + negl(n) $

Il distinguisher $D$ <b>sfrutta</b> l'attaccante efficiente $M$ dell'esperimento $PrivK^{eav}_{M,\ _pOTP}(n)$ ovvero ci gioca contro (impersona $A$ e quindi scriviamo $D(A)$, ma non possiamo assolutamente controllare $M$)

$PrivK^{eav}_{M,\ _pOTP}(n)$:

$M \rightarrow D(A): \quad \quad x_0, \quad x_1 \quad$ <i>non possiamo sceglierli noi perchè non siamo $M$, ma $D(A)$\
$D(A) \rightarrow M: \quad \quad y = w_{b_{PRG}} ⊕ x_{b_{PrivK}} \quad$ <i>il nostro input (da dinstinguisher) è $w_{b_{PRG}}$ e lo usiamo nell'encryption del messaggio di $M$: $x_{b_{PrivK}}$</i> \
$M: \quad \quad b_m \quad$ <i>$M$ restituisce il risultato della sua strategia contro lo pseudo-OTP</i> 

$se \ \ b_{PRG}=0: \ \ w_{b_{PRG}}=r \implies OTP \implies Pr(b_m = b_{PrivK}) = 1/2$\
$se \ \ b_{PRG}=1: \ \ w_{b_{PRG}}=G(k) \implies _pOTP \implies Pr(b_m = b_{PrivK}) > 1/2 + negl(n) \quad$ <i>per ipotesi<i>

Nel caso in cui $b_{PRG}$ sia 0, $M$ sta giocando contro l'$OTP$ e quindi ha 1/2 di probabilità di vincere.\
Nel caso in cui $b_{PRG}$ sia 1, $M$ sta giocando contro lo pseudo-$OTP$ che per ipotesi <b>non</b> è EAV-sicuro: probabilità di vincere è maggiore di $1/2 + negl(n)$\
È piu probabile che $M$ indovini quando $b_{PRG}$ è uguale a 1 (pseudo-OTP)

Cosa succede adesso nell'esperimento $PRG$?

$PRG_{D,G}(n):$

$1. \ A: \quad w_0 = r \leftarrow \{0,1\}^{l(n)} \quad \quad w_1=G(s) \ \ s \leftarrow \{0,1\}^n$ \
$2. \ A \rightarrow D: \quad w_{b_{PRG}} \quad \quad b_{PRG} \leftarrow \{0,1\}$ \
$3. \ D: \quad se \ \ b_m = b_{PrivK} \implies b_D = 1$

Nel passo 3 del $PRG$-experiment, $D$ sfrutta l'attaccante $M$ (che rende lo pseudo-OTP non sicuro) dell'esperimento $PrivK$.\
Allora $D$ si chiede: $M$ ha vinto ($b_m=b_{PrivK}$)? E allora dice 1 (ovvero pseudorandomness).

$Pr(PRG_{D,G}(n)=1) = \frac{1}{2}(Pr(b_D=0 | b_{PRG}=0) + Pr(b_D=1 | b_{PRG}=1))$

$Pr(b_D=0 | b_{PRG}=0) = Pr(b_m \neq b_{PrivK}\ | \ OTP) = 1/2 \quad$ <i>$D$ dice 0 quando $b_m \neq b_{PrivK}$ nell'esperimento PrivK e, essendo $OTP$, la probabilità è 1/2</i>\
$Pr(b_D=1 | b_{PRG}=1) = Pr(b_m = b_{PrivK}\ | \ _pOTP) > 1/2 + negl(n) \quad$ <i>per ipotesi</i>

$Pr(PRG_{D,G}(n)=1) > \frac{1}{2}(\frac{1}{2} + \frac{1}{2}+negl(n)) = \frac{1}{2} + negl(n) \implies$ $G$ <b>non</b> è PRG

Nota: $M \in PPTIME \implies D \in PPTIME$

### Esercizio

<div>
<img src="images/eav_security_ex2.png" width="500"/>
</div>

$PrivK^{eav}_{M,π}(n)$:

$1. \ A: \quad b \leftarrow\{0,1\} \quad \quad k \leftarrow Gen(1^n)$\
$2. \ M \rightarrow A: \quad x_0=0^{n+1} \quad x_1=1^{n+1}$ \
$3. \ A \rightarrow M: \quad y = E_k(x_b) \quad y=y_1..y_n,y_{n+1_{1}}..y_{n+1_{n+1}}$\
$4. \ M: \quad b_m = \{r = y[1..y_n] \ \ (y_{n+1_{1}}..y_{n+1_{n+1}} == G(r) ⊕ x_0) \ \ ? \ \ 0 \ \ : \ \ 1\}$

$r$ è visibile in chiaro in $y$, quindi $M$ può calcolare $G(r)$ (principio di Kerckhoffs: $G$ è noto a $M$, gli stiamo regalando il <b>seed</b> $r$)\
Se xorando $G(r)$ con $x_0$ ottiene la seconda parte del ciphertext, allora dirà 0

$M$ <b>vince sempre</b> $\rightarrow$ <b>non</b> è $EAV$-sicuro

# $PRF$ (Pseudorandom functions)

Abbiamo visto il teorema: \
$G$ è $PRG \iff$ $_pOTP \ $ è $\ EAV$-sicuro

Noi vogliamo però almeno la $CPA$-security

$PrivK^{CPA}_{M,π}(n)$:

$1. \ A: \quad b \leftarrow\{0,1\} \quad \quad k \leftarrow Gen(1^n)$\
$2. \ M \rightarrow A: \quad x_0 \quad x_1 \quad \quad con \ \ |x_0|=|x_1|$ \
$3. \ A \rightarrow M: \quad y = E_k(x_b) \quad \quad$ <i>$M$ ha <b>oracle access</b> a $E_k$</i>\
$4. \ M: \quad b_m$

$PrivK^{CPA}_{M,π}(n) = 1 \ \ if \ \ b_m = b$

## Funzione pseudorandom:

Abbiamo visto in precedenza i generatori pseudorandom ($G$), algoritmi <b>deterministici</b> che <b>espandono</b> un seed $k$ random e nessun distinguisher/attaccante <b>efficiente</b> è in grado di romperli (ovvero discriminare l'output del generatore da una sequenza effettivamente random) trascurando le probabilità negligible. 

$F \in \{0,1\}^{l_{key}(n)} \rightarrow (\{0,1\}^{l_{in}(n)} \rightarrow \{0,1\}^{l_{out}(n)})$

a meno di specificare il contrario assumiamo che: $\quad l_{key}(n)$ = $l_{in}(n)$ = $l_{out}(n) = n$

$F \in \{0,1\}^{n} \rightarrow (\{0,1\}^{n} \rightarrow \{0,1\}^n) \quad \quad F \in PTIME \quad \quad$ <i><b>ATTENZIONE:</b> $F$ è una <b>funzione</b> e quindi è <b>deterministica</b></i>

A differenza dei PRG, le funzioni pseudorandom non hanno il vincolo di espansività.

La differenza sostanziale è:\
$PRG$ genera <b>sequenze</b> pseudorandom\
$PRF$ genera <b>funzioni</b> pseudorandom

## Esperimento PRF:

$PRF_{D,F}(n)$:

$1. \ A: \quad f_0 \leftarrow (\{0,1\}^n \rightarrow \{0,1\}^n) \quad f_1 = F_k \quad \quad dove \ \ k \leftarrow \{0,1\}^n \quad b \leftarrow \{0,1\}$\
$2. \ D$ ha <b>oracle access</b> a $f_b \quad$ <i>questo per ovviare al problema della complessità esponenziale di $f_0$</i>\
$3. \ D: \quad b_d$

$PRF_{D,F}(n) = 1 \ \ se \ \ b_d = b$

Nel passo 1, in base al risultato di $b$, $A$ sceglie una <b>funzione</b>:\
$se \ \ b = 0: \ \ $ sceglie <i>letteralmente</i> una funzione a caso tra tutte le funzioni da $n$ bit a $n$ bit \
$se \ \ b = 1: \ \ $ utilizza la funzione $F$ applicata a $k$

$F$ è una <b>pseudorandom function</b> se $\quad \forall D \in PPTIME : \ \ Pr(PRF_{D,F}(n) = 1) \leq 1/2 + negl(n)$

## Esercizio: 

$F_k = x \ \& \ k \quad \quad$ $ \& \ = \ $ <i>bitwise AND</i> 

$F_k$ è una PRF?

$PRF_{D,F}(n)$:

$1. \ A: \quad f_0 \leftarrow (\{0,1\}^n \rightarrow \{0,1\}^n) \quad f_1 = F_k \quad \quad dove \ \ k \leftarrow \{0,1\}^n \quad b \leftarrow \{0,1\}$

$2. \ D$ usa $f_b \ $ in modalità <b>oracle</b> 

$D: \quad f_k(0^n)$ 

$3. \ D: \quad b_d \ = \ (y==0^n) \ ? \ 1 \ : \ 0$

$D$ è chiamato a scegliere una $x$ quando usa la funzione in modalità oracle; dal momento che $D$ sa come funziona $F$: manda come input una sequenza di 0 (0 & qualcosa = 0): se il risultato restituito da $f_b$ è $0^n$, la sua strategia è quella di dire 1 ($F_k$), altrimenti dice 0

$Pr(PRF_{D,F}(n) = 1) = \frac{1}{2}(Pr(b_d=0 | b=0) \cdot Pr(b_d=1 | b=1))$

$Pr(b_d=1 | b=1)\ = \ 1 \quad$ <i>vince sempre</i>\
$Pr(b_d=0 | b=0) = 1 - Pr(b_d=1 | b=0)$

$Pr(b_d=1 | b=0)$ = probabilità che la funzione random restituisca $0^n$ (quindi la probabilità che $D$ dica 1 in maniera errata in quanto la funzione è effettivamente random e lui pensava fosse l'and di k e $0^n$)

## Quante sono le funzioni in $(\{0,1\}^n \rightarrow \{0,1\}^n)$?

Rappresentiamo le funzioni (<b>non è</b> richiesto che sia iniettiva) come sequenza (in un array) dei loro output:

$input \rightarrow output$

$n=2$:\
$00 \rightarrow x_1x_2$ \
$01 \rightarrow x_3x_4$ \
$10 \rightarrow x_5x_6$ \
$11 \rightarrow x_7x_8$ 

bitstring: $\quad x_1x_2 \ x_3x_4 \ x_5x_6 \ x_7x_8 \quad \rightarrow 2^8$ possibili bitstrings (8 bit necessari per la rappresentazione)

$\quad 00000000 \quad 00000001 \quad 00000010 \quad 00000011 \quad ... \quad 11111110 \quad 11111111$

$n=3$:\
24 bit $\rightarrow$ $2^3 \cdot 3 = 24$ 

$n=4$:\
64 bit $\rightarrow$ $2^4 \cdot 4 = 64$ 

$n$ generico:\
$2^n \ $ blocchi da $n$ bit\
$2^n \cdot n \ $ bit

$|Fun_n| = 2^{2^n \cdot n}$

Insieme di tutte le funzioni $n$ bit lo rappresentiamo come $Fun_n \ $:

$|Fun_2| = 2^8$\
$|Fun_3| = 2^{24}$

## Esercizio (continuo)
 
Nell'esercizio sopra dovevamo calcolare: 

$Pr(b_d=1 | b=0)$ = probabilità che la funzione random restituisca $0^n$